In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Desigualdade CHSH

*Estimativa de uso: Dois minutos em um processador Heron r2 (NOTA: Esta é apenas uma estimativa. Seu tempo de execução pode variar.)*
## Contexto
Neste tutorial, você executará um experimento em um computador quântico para demonstrar a violação da desigualdade CHSH com a primitiva Estimator.

A desigualdade CHSH, nomeada em homenagem aos autores Clauser, Horne, Shimony e Holt, é usada para provar experimentalmente o teorema de Bell (1969). Este teorema afirma que teorias de variáveis ocultas locais não podem explicar algumas consequências do emaranhamento na mecânica quântica. A violação da desigualdade CHSH é usada para mostrar que a mecânica quântica é incompatível com teorias de variáveis ocultas locais. Este é um experimento importante para entender os fundamentos da mecânica quântica.

O Prêmio Nobel de Física de 2022 foi concedido a Alain Aspect, John Clauser e Anton Zeilinger, em parte por seu trabalho pioneiro em ciência da informação quântica e, em particular, por seus experimentos com fótons emaranhados demonstrando a violação das desigualdades de Bell.
## Requisitos
Antes de começar este tutorial, certifique-se de ter o seguinte instalado:

* Qiskit SDK v1.0 ou posterior, com suporte para [visualização](https://docs.quantum.ibm.com/api/qiskit/visualization)
* Qiskit Runtime (`pip install qiskit-ibm-runtime`) v0.22 ou posterior
## Configuração

In [1]:
# General
import numpy as np

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Qiskit Runtime imports
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator

# Plotting routines
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

## Passo 1: Mapear entradas clássicas para um problema quântico
Para este experimento, criaremos um par emaranhado no qual mediremos cada qubit em duas bases diferentes. Rotularemos as bases para o primeiro qubit como $A$ e $a$ e as bases para o segundo qubit como $B$ e $b$. Isso nos permite calcular a quantidade CHSH $S_1$:

$$
S_1 = A(B-b) + a(B+b).
$$

Cada observável é $+1$ ou $-1$. Claramente, um dos termos $B\pm b$ deve ser $0$, e o outro deve ser $\pm 2$. Portanto, $S_1 = \pm 2$. O valor médio de $S_1$ deve satisfazer a desigualdade:

$$
|\langle S_1 \rangle|\leq 2.
$$

Expandindo $S_1$ em termos de $A$, $a$, $B$ e $b$ resulta em:

$$
|\langle S_1 \rangle| = |\langle AB \rangle - \langle Ab \rangle + \langle aB \rangle + \langle ab \rangle| \leq 2
$$

Você pode definir outra quantidade CHSH $S_2$:

$$
S_2 = A(B+b) - a(B-b),
$$

Isso leva a outra desigualdade:

$$
|\langle S_2 \rangle| = |\langle AB \rangle + \langle Ab \rangle - \langle aB \rangle + \langle ab \rangle| \leq 2
$$

Se a mecânica quântica puder ser descrita por teorias de variáveis ocultas locais, as desigualdades anteriores devem ser verdadeiras. No entanto, como será demonstrado neste tutorial, essas desigualdades podem ser violadas em um computador quântico. Portanto, a mecânica quântica não é compatível com teorias de variáveis ocultas locais.
Se você quiser aprender mais teoria, explore [Emaranhamento em Ação](/learning/courses/basics-of-quantum-information/entanglement-in-action/chsh-game) com John Watrous.
Você criará um par emaranhado entre dois qubits em um computador quântico criando o estado de Bell $|\Phi^+\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}}$. Usando a primitiva Estimator, você pode obter diretamente os valores esperados necessários ($\langle AB \rangle, \langle Ab \rangle, \langle aB \rangle$ e $\langle ab \rangle$) para calcular os valores esperados das duas quantidades CHSH $\langle S_1\rangle$ e $\langle S_2\rangle$. Antes da introdução da primitiva Estimator, você teria que construir os valores esperados a partir dos resultados das medições.

Você medirá o segundo qubit nas bases $Z$ e $X$. O primeiro qubit também será medido em bases ortogonais, mas com um ângulo em relação ao segundo qubit, que variaremos entre $0$ e $2\pi$. Como você verá, a primitiva Estimator torna a execução de circuitos parametrizados muito fácil. Em vez de criar uma série de circuitos CHSH, você só precisa criar *um* circuito CHSH com um parâmetro especificando o ângulo de medição e uma série de valores de fase para o parâmetro.

Finalmente, você analisará os resultados e os plotará em função do ângulo de medição. Você verá que para certos intervalos de ângulos de medição, os valores esperados das quantidades CHSH $|\langle S_1\rangle| > 2$ ou $|\langle S_2\rangle| > 2$, o que demonstra a violação da desigualdade CHSH.

In [2]:
# To run on hardware, select the backend with the fewest number of jobs in the queue
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
backend.name

'ibm_kingston'

### Create a parameterized CHSH circuit

First, we write the circuit with the parameter $\theta$, which we call `theta`. The [`Estimator` primitive](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) can enormously simplify circuit building and output analysis by directly providing expectation values of observables. Many problems of interest, especially for near-term applications on noisy systems, can be formulated in terms of expectation values. `Estimator` (V2) primitive can automatically change measurement basis based on the supplied observable.

In [3]:
theta = Parameter("$\\theta$")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)
chsh_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif" alt="Output of the previous code cell" />

### Criar um circuito CHSH parametrizado
Primeiro, escrevemos o circuito com o parâmetro $\theta$, que chamamos de `theta`. A [primitiva `Estimator`](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) pode simplificar enormemente a construção de circuitos e a análise de saída, fornecendo diretamente valores esperados de observáveis. Muitos problemas de interesse, especialmente para aplicações de curto prazo em sistemas ruidosos, podem ser formulados em termos de valores esperados. A primitiva `Estimator` (V2) pode automaticamente mudar a base de medição com base no observável fornecido.

In [4]:
number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
# Phases need to be expressed as list of lists in order to work
individual_phases = [[ph] for ph in phases]

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif)

### Criar uma lista de valores de fase a serem atribuídos posteriormente
Após criar o circuito CHSH parametrizado, você criará uma lista de valores de fase a serem atribuídos ao circuito na próxima etapa. Você pode usar o código a seguir para criar uma lista de 21 valores de fase variando de $0$ a $2 \pi$ com espaçamento igual, ou seja, $0$, $0.1 \pi$, $0.2 \pi$, ..., $1.9 \pi$, $2 \pi$.

In [5]:
# <CHSH1> = <AB> - <Ab> + <aB> + <ab> -> <ZZ> - <ZX> + <XZ> + <XX>
observable1 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", -1), ("XZ", 1), ("XX", 1)]
)

# <CHSH2> = <AB> + <Ab> - <aB> + <ab> -> <ZZ> + <ZX> - <XZ> + <XX>
observable2 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", 1), ("XZ", -1), ("XX", 1)]
)

### Observáveis
Agora precisamos de observáveis a partir dos quais calcular os valores esperados. No nosso caso, estamos observando bases ortogonais para cada qubit, deixando a rotação $Y$ parametrizada para o primeiro qubit varrer a base de medição quase continuamente em relação à base do segundo qubit. Portanto, escolheremos os observáveis $ZZ$, $ZX$, $XZ$ e $XX$.

In [6]:
target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

chsh_isa_circuit = pm.run(chsh_circuit)
chsh_isa_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif" alt="Output of the previous code cell" />

## Passo 2: Otimizar o problema para execução em hardware quântico

Para reduzir o tempo total de execução do trabalho, as primitivas V2 aceitam apenas circuitos e observáveis que estejam em conformidade com as instruções e conectividade suportadas pelo sistema de destino (referidos como circuitos e observáveis de arquitetura de conjunto de instruções (ISA)).

### Circuito ISA

In [7]:
isa_observable1 = observable1.apply_layout(layout=chsh_isa_circuit.layout)
isa_observable2 = observable2.apply_layout(layout=chsh_isa_circuit.layout)

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif)

### Observáveis ISA

Da mesma forma, precisamos transformar os observáveis para torná-los compatíveis com o backend antes de executar trabalhos com o [`Runtime Estimator V2`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2#run). Podemos realizar a transformação usando o método `apply_layout` do objeto `SparsePauliOp`.

In [8]:
# To run on a local simulator:
# Use the StatevectorEstimator from qiskit.primitives instead.

estimator = Estimator(mode=backend)

pub = (
    chsh_isa_circuit,  # ISA circuit
    [[isa_observable1], [isa_observable2]],  # ISA Observables
    individual_phases,  # Parameter values
)

job_result = estimator.run(pubs=[pub]).result()

## Passo 3: Executar usando primitivas Qiskit
Para executar todo o experimento em uma única chamada ao [`Estimator`](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2).
Podemos criar uma primitiva [`Estimator` do Qiskit Runtime](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2) para calcular nossos valores esperados. O método `EstimatorV2.run()` recebe um iterável de `blocos unificados primitivos (PUBs)`. Cada PUB é um iterável no formato `(circuit, observables, parameter_values: Optional, precision: Optional)`.

In [9]:
chsh1_est = job_result[0].data.evs[0]
chsh2_est = job_result[0].data.evs[1]

In [10]:
fig, ax = plt.subplots(figsize=(10, 6))

# results from hardware
ax.plot(phases / np.pi, chsh1_est, "o-", label="CHSH1", zorder=3)
ax.plot(phases / np.pi, chsh2_est, "o-", label="CHSH2", zorder=3)

# classical bound +-2
ax.axhline(y=2, color="0.9", linestyle="--")
ax.axhline(y=-2, color="0.9", linestyle="--")

# quantum bound, +-2√2
ax.axhline(y=np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.axhline(y=-np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.fill_between(phases / np.pi, 2, 2 * np.sqrt(2), color="0.6", alpha=0.7)
ax.fill_between(phases / np.pi, -2, -2 * np.sqrt(2), color="0.6", alpha=0.7)

# set x tick labels to the unit of pi
ax.xaxis.set_major_formatter(tck.FormatStrFormatter("%g $\\pi$"))
ax.xaxis.set_major_locator(tck.MultipleLocator(base=0.5))

# set labels, and legend
plt.xlabel("Theta")
plt.ylabel("CHSH witness")
plt.legend()
plt.show()

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/f6267448-0.avif" alt="Output of the previous code cell" />

In the figure, the lines and gray areas delimit the bounds; the outer-most (dash-dotted) lines delimit the quantum-bounds ($\pm 2$), whereas the inner (dashed) lines delimit the classical bounds ($\pm 2\sqrt{2}$). You can see that there are regions where the CHSH witness quantities exceeds the classical bounds. Congratulations! You have successfully demonstrated the violation of CHSH inequality in a real quantum system!

## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_3xxAgm1SF1wGp9k)